In [1]:
import os
import time
import re
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# --- إعداد Selenium ---
chromedriver_path = r'C:\Users\pc\Desktop\scrp\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)

chrome_options = Options()
chrome_options.add_argument(r"user-data-dir=C:\Users\pc\AppData\Local\Google\Chrome\User Data")
chrome_options.add_argument("profile-directory=Default")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=service, options=chrome_options)

# --- الانتقال إلى موقع ChatGPT ---
url = "https://chatgpt.com/"
driver.get(url)
time.sleep(5)  # تعديل الوقت حسب الحاجة

# --- دالة إرسال الطلب واستلام الرد ---
def get_valid_response(msg, data_content):
    """
    تُكوّن الدالة الاستعلام من msg و data_content، وترسله عبر ChatGPT،
    ثم تنتظر لمدة 40 ثانية، وتقوم بالتمرير لأسفل الصفحة وتحديد آخر رد
    (الموجود داخل عنصر الكود) لتحصل على الرد الخاص بالطلب الحالي.
    """
    query = f"{msg}\n{data_content}"
    print("إرسال الاستعلام التالي:")
    print(query)
    
    # العثور على حقل الإدخال وتنظيفه
    chat_input = driver.find_element(By.ID, "prompt-textarea")
    driver.execute_script("arguments[0].innerHTML = '';", chat_input)
    time.sleep(1)
    
    # إدخال النص في الحقل وإرسال الاستعلام
    driver.execute_script("arguments[0].focus(); document.execCommand('insertText', false, arguments[1]);", chat_input, query)
    time.sleep(4)
    chat_input.send_keys(Keys.RETURN)
    
    # الانتظار حتى يُكتب الرد بالكامل
    time.sleep(40)
    
    # التمرير لأسفل الصفحة لضمان ظهور الردود الجديدة
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    # جلب جميع عناصر الردود الموجودة داخل div يحتوي على كود HTML
    responses = driver.find_elements(By.CSS_SELECTOR, 'div.overflow-y-auto.p-4 code')
    if not responses:
        print("لم يتم العثور على أي رد.")
        return ""
    
    # اختيار آخر رد للحصول على الرد الخاص بالطلب الحالي
    response_elem = responses[-1]
    translated_content = response_elem.get_attribute('innerText')
    print("الرد الذي تم استلامه:")
    print(translated_content)
    
    # التحقق من صحة تنسيق الرد (بداية <!DOCTYPE html> ونهاية </html>)
    if translated_content.strip().startswith("<!DOCTYPE html>") and translated_content.strip().endswith("</html>"):
        return translated_content
    else:
        print("الرد غير صالح، يتم إعادة إرسال الطلب...")
        return get_valid_response(msg, data_content)

# --- إعداد ملف تتبع الملفات المعالجة ---
dossier_principal = "repas"
checkpoint_file = os.path.join(dossier_principal, "processed_files.json")
processed_files = {}

# تحميل البيانات السابقة إن وجدت
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r", encoding="utf-8") as cp:
        processed_files = json.load(cp)
else:
    processed_files = {}  # سنستخدم dict بحيث المفتاح هو مسار الملف والقيمة حالة المعالجة

# --- قراءة الرسالة الأساسية من ملف text.txt داخل مجلد repas ---
fichier_msg = os.path.join(dossier_principal, "text.txt")
if os.path.exists(fichier_msg):
    with open(fichier_msg, "r", encoding="utf-8") as f:
        msg = f.read().strip()
else:
    print("ملف text.txt غير موجود في مجلد repas.")
    driver.quit()
    exit()

# --- معالجة المجلدات الفرعية والملفات ---
for dossier in os.listdir(dossier_principal):
    dossier_path = os.path.join(dossier_principal, dossier)
    if os.path.isdir(dossier_path):
        print(f"معالجة المجلد: {dossier}")
        # التكرار على جميع ملفات Excel في المجلد الفرعي
        for fichier in os.listdir(dossier_path):
            if fichier.endswith(".xlsx"):
                fichier_excel = os.path.join(dossier_path, fichier)
                
                # التحقق إذا كان الملف قد عُالج بالفعل
                if processed_files.get(fichier_excel, False):
                    print(f"تمت معالجة {fichier_excel} مسبقًا، يتم تخطيه.")
                    continue
                
                try:
                    df = pd.read_excel(fichier_excel)
                    data_content = df.to_json(orient='records', force_ascii=False)
                    # الحصول على الرد الصحيح عبر ChatGPT
                    translated_content = get_valid_response(msg, data_content)
                    
                    # استخراج عنوان الوصفة من وسم <title>
                    match = re.search(r"<title>(.*?)</title>", translated_content, re.IGNORECASE | re.DOTALL)
                    if match and match.group(1).strip():
                        titre = match.group(1).strip()
                        # حذف الأحرف غير الصالحة في اسم الملف إن وجدت
                        titre = "".join(c for c in titre if c.isalnum() or c in " -_").strip()
                    else:
                        print("لم يتم العثور على وسم <title>، سيتم استخدام اسم الملف الأصلي.")
                        titre = os.path.splitext(fichier)[0]
                    
                    # تحديد مسار حفظ ملف HTML باستخدام العنوان المستخرج
                    fichier_html = os.path.join(dossier_path, f"{titre}.html")
                    with open(fichier_html, "w", encoding="utf-8") as f:
                        f.write(translated_content)
                    
                    print(f"تم حفظ الرد في {fichier_html}")
                    
                    # تحديث ملف التحقق لحفظ أن هذا الملف قد عُالج
                    processed_files[fichier_excel] = True
                    with open(checkpoint_file, "w", encoding="utf-8") as cp:
                        json.dump(processed_files, cp, ensure_ascii=False, indent=4)
                
                except Exception as e:
                    print(f"حدث خطأ أثناء معالجة {fichier_excel}: {e}")
                    # يُمكن هنا تسجيل الخطأ في ملف سجل أو اتخاذ إجراءات أخرى
                    continue

# إغلاق المتصفح بعد انتهاء المعالجة
driver.quit()